In [1]:
import os
import glob
import pandas as pd

In [6]:
script_dir = os.path.dirname(os.path.realpath('dev.ipynb'))


In [11]:
last_date

'2024-07-01'

In [12]:
def normalize_series(series):

    mean, std = series.mean(),series.std()
    series = (series - mean)/std

    return series


In [17]:
[round(x,5) for x in normalize_series(pd.Series([-3, -1, 0, 1, 3])).to_list()]

[-1.34164, -0.44721, 0.0, 0.44721, 1.34164]

In [15]:
script_dir = os.path.dirname(os.path.realpath('dev.ipynb'))

In [35]:
output = pd.read_csv(f'{script_dir}/../output_data/tableau.csv')

In [36]:
output

,Date,Redfin HPI MoM,Type
0,2012-04-01,1.730000,Series
1,2012-05-01,1.560000,Series
2,2012-06-01,1.930000,Series
3,2012-07-01,0.880000,Series
4,2012-08-01,1.470000,Series
...,...,...,...
147,2024-07-01,0.380000,Series
148,2024-07-01,0.380000,Forecast
149,2024-08-01,0.056024,Forecast
150,2024-09-01,0.541787,Forecast


In [37]:
current_metrics = pd.read_csv(f'{script_dir}/../output_data/model_metrics.csv')


In [38]:
current_metrics

,Date,Pred Date,Forecast Pred,MAE
0,20240701,2024-06-01,0.583043,0.584990
1,20240801,2024-07-01,0.953691,0.622643
2,20240901,2024-08-01,0.056024,0.640131
3,20240915,2024-08-01,0.056024,0.640131
4,20240915,2024-08-01,0.056024,0.640131
5,20240915,2024-08-01,0.056024,0.640131
6,20240915,2024-08-01,0.056024,0.640131
7,20240915,2024-08-01,0.056024,0.640131


In [32]:
current_metrics = pd.read_csv(f'{script_dir}/../output_data/model_metrics.csv')
current_metrics = current_metrics[['Pred Date','Forecast Pred']]
current_metrics.rename(columns={'Pred Date':'Date'},inplace=True)


In [34]:
current_metrics.merge(output[['Date','Redfin HPI MoM']],on='Date',how='left')

,Date,Forecast Pred,Redfin HPI MoM
0,2024-06-01,0.583043,0.390000
1,2024-07-01,0.953691,0.380000
2,2024-07-01,0.953691,0.380000
3,2024-08-01,0.056024,0.056024


In [22]:
output

,Date,Redfin HPI MoM,Type
0,2012-04-01,1.730000,Series
1,2012-05-01,1.560000,Series
2,2012-06-01,1.930000,Series
3,2012-07-01,0.880000,Series
4,2012-08-01,1.470000,Series
...,...,...,...
147,2024-07-01,0.380000,Series
148,2024-07-01,0.380000,Forecast
149,2024-08-01,0.056024,Forecast
150,2024-09-01,0.541787,Forecast


In [70]:
with open(f'{script_dir}/../README.md', 'r') as file:
        readme_lines = file.readlines()

In [71]:
plots = glob.glob(f'{script_dir}/../forecast_plots/*forecast*')
plots = sorted([x for x in plots if 'zoomed' not in x])
zoomed_plots = sorted(glob.glob(f'{script_dir}/../forecast_plots/*zoomed*'))

In [72]:
latest_zoomed = zoomed_plots[-1].split('../')[1]
latest_normal = plots[-1].split('../')[1]

In [73]:
latest_normal

'forecast_plots/20240901_forecast.png'

In [74]:
for i, line in enumerate(readme_lines):
    if '![alt text]' in line:
        if 'zoomed' in line:
            new_line = f'![alt text]({latest_zoomed})\n'
            readme_lines[i] = new_line
        elif 'forecast.png' in line:
            new_line = f'![alt text]({latest_normal})\n'
            readme_lines[i] = new_line
        else:
            continue

In [75]:
readme_lines

["# Forecasting Redfin's Housing Price Index In Los Angeles\n",
 '\n',
 '### Background\n',
 '\n',
 'In 2023, Redfin released their Housing Price Index, which uses the repeat sales pricing method to measure changes in U.S home prices. More information about the index can be found at - https://www.redfin.com/news/redfin-home-price-index/. They release their index monthly, with about a 2 month lag. In this project, I use various time series forecasting methods to forecast their index into the future, allowing interested parties such as real estate professionals, buyers, sellers, and financial institutions to look ahead at what is coming in the housing market. \n',
 '\n',
 'I decided to perform this analysis only on the Los Angeles Area to keep my analysis more focused. \n',
 '\n',
 '![alt text](forecast_plots/original_series.png)\n',
 '\n',
 '### Methods and Data\n',
 '\n',
 'To forecast this time series, I tried using both arima methods and vector autoregression methods. Ultimately, the

In [4]:
readme_lines

["# Forecasting Redfin's Housing Price Index In Los Angeles\n",
 '\n',
 '### Background\n',
 '\n',
 'In 2023, Redfin released their Housing Price Index, which uses the repeat sales pricing method to measure changes in U.S home prices. More information about the index can be found at - https://www.redfin.com/news/redfin-home-price-index/. They release their index monthly, with about a 2 month lag. In this project, I use various time series forecasting methods to forecast their index into the future, allowing interested parties such as real estate professionals, buyers, sellers, and financial institutions to look ahead at what is coming in the housing market. \n',
 '\n',
 'I decided to perform this analysis only on the Los Angeles Area to keep my analysis more focused. \n',
 '\n',
 '![alt text](forecast_plots/original_series.png)\n',
 '\n',
 '### Methods and Data\n',
 '\n',
 'To forecast this time series, I tried using both arima methods and vector autoregression methods. Ultimately, the